In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install image-classifiers==0.2.2
!pip install git+https://github.com/qubvel/classification_models.git

     |████████████████████████████████| 81kB 5.6MB/s 
  Cloning https://github.com/qubvel/classification_models.git to /tmp/pip-req-build-7bnwstxz
  Running command git clone -q https://github.com/qubvel/classification_models.git /tmp/pip-req-build-7bnwstxz
  Running command git submodule update --init --recursive -q
     |████████████████████████████████| 51kB 4.3MB/s 
  Created wheel for image-classifiers: filename=image_classifiers-1.0.0-cp36-none-any.whl size=20031 sha256=13cf273466cdbafce36225e56e49529a22b39d422748d577b4f4c60d633cdca5
  Stored in directory: /tmp/pip-ephem-wheel-cache-15ntzgqc/wheels/de/2b/fd/29a6d33edb8c28bc7d94e95ea1d39c9a218ac500a3cfb1b197
Successfully built image-classifiers
  Found existing installation: image-classifiers 0.2.2
    Uninstalling image-classifiers-0.2.2:
      Successfully uninstalled image-classifiers-0.2.2


In [ ]:
BS = 128
DROPOUT_RATE = 0.5
FROZEN_LAYER_NUM = 170

ADAM_LEARNING_RATE = 0.001
SGD_LEARNING_RATE = 0.01
SGD_DECAY = 0.0001

Resize_pixelsize = 197

In [ ]:
from keras import backend as K
def svm_loss(layer):
    weights = layer.weights[0]
    weights_tf = tf.convert_to_tensor(weights)
    
    def categorical_hinge_loss(y_true, y_pred):
        pos = K.sum(y_true * y_pred, axis=-1)
        neg = K.max((1.0 - y_true) * y_pred, axis=-1)
        hinge_loss = K.mean(K.maximum(0.0, neg - pos + 1), axis=-1)
        regularization_loss = 0.5*(tf.reduce_sum(tf.square(weights_tf)))
        return regularization_loss + 0.4*hinge_loss
    
    return categorical_hinge_loss

In [ ]:
import keras
import tensorflow as tf
from classification_models.keras import Classifiers

ResNet18, preprocess_input = Classifiers.get('resnet18')
# build model
n_classes = 7
base_model = ResNet18(input_shape=(197,197,3), weights='imagenet', include_top=False)
x = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(7, use_bias=False, activation='linear', name='svm')(x)
model = keras.models.Model(inputs=[base_model.input], outputs=[output])

# train
model.compile(optimizer='SGD', loss=svm_loss(model.get_layer('svm')), metrics=['accuracy'])

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def load_data(path, split=False):
    if split:
      datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
      return datagen.flow_from_directory(
              path,
              target_size=(197, 197),
              shuffle = True,
              class_mode='categorical',
              batch_size=BS,
              subset='training'), datagen.flow_from_directory(
              path,
              target_size=(197, 197),
              shuffle = True,
              class_mode='categorical',
              batch_size=BS,
              subset='validation')
    else:
      datagen = ImageDataGenerator(rescale=1./255)
      return datagen.flow_from_directory(
              path,
              target_size=(197, 197),
              shuffle = True,
              class_mode='categorical',
              batch_size=BS), None

In [ ]:
train_generator, validation_generator  = load_data('/content/drive/MyDrive/dataset/front', True)
test_generator, _  = load_data('/content/drive/MyDrive/dataset/test')

Found 16044 images belonging to 7 classes.
Found 4008 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [ ]:
rlrop = keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy',mode='max',factor=0.5, patience=10, min_lr=0.00001, verbose=1)
history = model.fit_generator(
    generator = train_generator,
    validation_data=validation_generator, 
    steps_per_epoch=16044 // BS,
    validation_steps=4008 // BS,
    shuffle=True,
    epochs=100,
    callbacks=[rlrop],
    # use_multiprocessing=True,
)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/100
125/125 [==============================] - 10010s 80s/step - loss: 7.3294 - accuracy: 0.3287 - val_loss: 7.4396 - val_accuracy: 0.2800
Epoch 2/100
125/125 [==============================] - 40s 316ms/step - loss: 7.1837 - accuracy: 0.4639 - val_loss: 7.3036 - val_accuracy: 0.1772
Epoch 3/100
125/125 [==============================] - 41s 325ms/step - loss: 7.1186 - accuracy: 0.5381 - val_loss: 7.3283 - val_accuracy: 0.1628
Epoch 4/100
125/125 [==============================] - 42s 333ms/step - loss: 7.0759 - accuracy: 0.5998 - val_loss: 7.3405 - val_accuracy: 0.2644
Epoch 5/100
125/125 [==============================] - 41s 330ms/step - loss: 7.0412 - accuracy: 0.6465 - val_loss: 7.3290 - val_accuracy: 0.3160
Epoch 6/100
125/125 [==============================] - 41s 331ms/step - loss: 7.0156 - accuracy: 0.6871 - val_loss: 7.2882 - val_accuracy: 0.3538
Epoch 7/100
125/125 [==========================

In [ ]:
results_validation = model.evaluate_generator(validation_generator)

Instructions for updating:
Please use Model.evaluate, which supports generators.


In [ ]:
results_test = model.evaluate_generator(test_generator)
print('test loss, test acc:', results_test)

test loss, test acc: [7.539792060852051, 0.3871552050113678]


In [ ]:
model.save('/content/drive/My Drive/' + 'RESNET18' + '.h5')